<a href="https://colab.research.google.com/github/Giuseppe86-lab/Software-Gestione-Magazzino/blob/main/Soft_Gest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import json
import os

class Wms():
  """
  Questa classe definisce le operazioni che possono essere fatte dal software
  di gestione del magazzino.
  """
  def __init__(self, inventory, _lordo = 0, _netto = 0):
      """
        Inizializza l'istanza Wms.

        Metodi:
        - select_new_operation(self, AVEILABLE_COMMAND)
        - add_product(self)
        - list_products(self)
        - sell_product(self)
        - profits(self)
        - delete_product(self, product)
        - help(self)
        - close(self)


        Argomenti:
        - inventory (dict): dizionario che contiene i prodotti come chiavi e
        come valori la quantità, il prezzo di acquisto e il prezzo di vendita.
        - _lordo (float): ricavo delle vendite dei prodotti del magazzino, valore
        di default zero.
        - _netto (float): guadagno delle vendite dei prodotti del magazzino,
        valore di default zero.
      """

      self.inventory = inventory
      self._lordo = _lordo
      self._netto = _netto

  def select_new_operation(self, AVEILABLE_COMMAND):
    """
    Chiede all'utente di selezionare un'operazione e verifica che sia tra quelle
    definite nei metodi della classe. Se l'input inserito non è presente in lista
    stampa un messaggio di errore.

    Argomenti:
    - AVEILABLE_COMMAND (list): lista con tutti i comandi eseguibili dall'utente.
    """
    try:
      command = input("\nInserisci un comando: ")
      assert(command in AVEILABLE_COMMAND), 'Comando non valido'
      return command
    except AssertionError as e:
      print(e)
      self.help()

  def add_product(self):
    """
    Chiede all'utente se vuole aggiungere un prodotto, se il prodotto non è presente
    nel magazzino, verrà richiesto oltre alla quantità il prezzo di acquisto e di
    vendita. Se il prodotto è già presente in magazzino, verrà richiesta solo la
    quantità.

    La funzione controlla che le quantità siano dei numeri interi positivi e i
    prezzi dei numeri positivi.
    """
    product = input('Nome del prodotto: ')
    if product.lower() in self.inventory.keys():
      try:
        amount = int(input('Quantità: '))
        assert (amount >= 0), "la quantità dev'essere un numero intero positivo"
        self.inventory[product.lower()]['Quantità'] += amount
        print(f"AGGIUNTO: {amount} X {product.lower()}")
      except ValueError:
        print("Impossibile aggiungere il prodotto: la quantità dev'essere"
              " un numero intero")
      except AssertionError as e:
        print(e)

    else:
      try:
        amount = int(input('Quantità: '))
        assert (amount >=0), "la quantità dev'essere un numero intero positivo"
        cost = float(input('Prezzo di acquisto: '))
        assert (cost >= 0), "il prezzo di acquisto dev'essere un numero positivo"
        price = float(input('Prezzo di vendita: '))
        assert (price >= 0), "il prezzo di vendita dev'essere un numero positivo"

        self.inventory[product.lower()] = {}
        self.inventory[product.lower()]['Quantità'] = amount
        self.inventory[product.lower()]['Prezzo di acquisto'] = cost
        self.inventory[product.lower()]['Prezzo di vendita'] = price
        print(f"AGGIUNTO: {self.inventory[product.lower()]['Quantità']} X {product.lower()}")
      except ValueError:
        print("Impossibile aggiungere il prodotto: La quantità dev'essere"
              "un numero intero positivi, mentre i prezzi dei numeri positivi")
      except AssertionError as e:
        print(e)


  def list_products(self):
    """
    Stampa una tabella con la lista dei prodotti presenti in magazzino, la
    quntità e il prezzo di vendita.
    Se non ci sono prodotti presenti in magazzino, stampa un messaggio di errore.
    """
    try:
      max_length_product_name = max(len(product) for product in self.inventory.keys())
      print(f"{'PRODOTTO':<{max_length_product_name+5}}\tQUANTITA'\tPREZZO")
      for product_name, product_info in self.inventory.items():
        print(f"{product_name:<{max_length_product_name+5}}\t{product_info['Quantità']:<10}\t€{product_info['Prezzo di vendita']:<10}")
    except ValueError:
      print('Il magazzino è vuoto')

  def sell_product(self):
    """
    Chiede all'utente quale prodotto vuole acquistare e la quantità, controlla se
    la merce è presente in magazzino e se la quantità richiesta è disponibile.

    Permette di comprare più prodotti uno dopo l'altro e alla fine stampa un
    resoconto delle merci acquistate e del prezzo pagato.

    Contemporaneamente aggiorna le disponibilità del magazzino. Se la merce è
    terminata chiama il metodo per eliminarla.
    """
    selling_on = 'si'
    record_selling = {}
    total_revenue = 0
    total_cost = 0
    while selling_on == 'si':

      product = input('Nome del prodotto: ')
      try:

        assert(product in self.inventory.keys()), 'Prodotto non presente in magazzino'
        n_product = int(input('Quantità: '))
        assert(n_product <= self.inventory[product]['Quantità']), f"Max disponibilità {self.inventory[product]['Quantità']}"
        record_selling[product]= [n_product, self.inventory[product]['Prezzo di vendita']]
        total_revenue += n_product*self.inventory[product]['Prezzo di vendita']
        total_cost += n_product*self.inventory[product]['Prezzo di acquisto']
        self.inventory[product]['Quantità'] -= n_product

        if self.inventory[product]['Quantità'] == 0:
          self.delete_product(product)

        selling_on = input('Aggiungere un altro prodotto ? (si/no):')

      except AssertionError as e:
        print(e)

      except ValueError:
        print("La quantità dev'essere un numero intero")

    self._lordo += total_revenue
    self._netto += total_revenue - total_cost
    print("VENDITA REGISTRATA")
    for product_name, product_info in record_selling.items():
      print(f'- {product_info[0]} X {product_name}: €{product_info[1]}')
    print(f'Totale: €{total_revenue:.2f}')

  def profits(self):
    """
    Stampa i profitti sia lordi che netti raggiunti durante l'attuale sessione
    di utilizza del software gestionale.
    """
    print(f"Profitto: lordo=€{self._lordo:.2f} netto=€{self._netto:.2f}")

  def delete_product(self, product):
    """
    Elimina prodotti dal dizionario del magazzino, elimina tutte le informazioni
    legate al prodotto.

    Argomenti:
    - product(str): nome del prodotto che si vuole eliminare.
    """
    del self.inventory[product]

  def help(self):
    """
    Stampa una lista di tutti i comandi disponibili, dandone una breve descrizione.
    """
    return print("I comandi disponibili sono i seguenti:\n"
    "aggiungi: aggiungi un prodotto al magazzino\n"
    "elenca: elenca i prodotti in magazzino\n"
    "vendita: registra una vendita effettuata\n"
    "profitti: mostra i profitti totali\n"
    "aiuto: mostra i possibili comandi\n"
    "chiudi: esci dal programma\n")

  def close(self):
    """
    Salva il dizionario con l'inventario in un file .json per sessioni successive.
    Stampa un messaggio di saluto.
    """
    with open("inventory_veg_shop.json", 'w', encoding="utf-8") as json_file:
      json.dump(self.inventory, json_file, ensure_ascii=False, indent = 6)
    print("Bye bye\n")


def load_inventory():
  """
  Cambia la directory in quella attuale di lavoro.

  Cerca nella directory di lavoro se è presente il file con i dati di una sessione
  precedente. Se è presente, legge il file .json, e restituisce il dizionario.
  Se non è presente inizializza un dizionario e lo restituisce.
  """
  new_folder = "/content/drive/MyDrive/Colab Notebooks"
  os.chdir(new_folder)
  try:
    with open("inventory_veg_shop.json", encoding="utf-8") as json_file:
      inventory = json.load(json_file)

  except FileNotFoundError:
    inventory = {}

  return inventory

"""
Questa parte rappresenta il main del software gestionale, definisce:
- la lista dei comandi accessibili;
- chiama la funzione per caricare l'inventario;
- inizializza l'oggetto Wms;
- chiede all'utente quale comando vuole eseguire e, se il comando è valido, chiama
i metodo corrispondenti.
"""

AVEILABLE_COMMAND = ['aggiungi', 'elenca', 'vendita', 'profitti', 'aiuto', 'chiudi']


opened_inventory = load_inventory()
my_inventory = Wms(opened_inventory)

closure = False

while not closure:
  command = my_inventory.select_new_operation(AVEILABLE_COMMAND)

  if command == 'aiuto':
    my_inventory.help()

  if command == 'aggiungi':
    my_inventory.add_product()

  if command == 'elenca':
    my_inventory.list_products()

  if command == 'vendita':
    my_inventory.sell_product()

  if command == 'profitti':
    my_inventory.profits()

  if command == 'chiudi':
    my_inventory.close()
    closure = True




Inserisci un comando: aiuto
I comandi disponibili sono i seguenti:
aggiungi: aggiungi un prodotto al magazzino
elenca: elenca i prodotti in magazzino
vendita: registra una vendita effettuata
profitti: mostra i profitti totali
aiuto: mostra i possibili comandi
chiudi: esci dal programma


Inserisci un comando: elenca
Il magazzino è vuoto

Inserisci un comando: aggiungi
Nome del prodotto: latte di soia
Quantità: 3
Prezzo di acquisto: 1.1
Prezzo di vendita: 2.1
AGGIUNTO: 3 X latte di soia

Inserisci un comando: tofu
Comando non valido
I comandi disponibili sono i seguenti:
aggiungi: aggiungi un prodotto al magazzino
elenca: elenca i prodotti in magazzino
vendita: registra una vendita effettuata
profitti: mostra i profitti totali
aiuto: mostra i possibili comandi
chiudi: esci dal programma


Inserisci un comando: aggiungi
Nome del prodotto: tofu
Quantità: 5
Prezzo di acquisto: 2.4
Prezzo di vendita: 3.9
AGGIUNTO: 5 X tofu

Inserisci un comando: aggiungi
Nome del prodotto: seitan
Quantità:

In [8]:
print(my_inventory)